# YOLOv5 on Custom Objects

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference

In [1]:
!pip install wandb -qqq
import wandb
# wandb.login()
# wandb.init(project="YOLOv5")

In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/lab03/EvolveYOLO/yolov5
HEAD is now at 886f1c0 DDP after autoanchor reorder (#2421)


In [4]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
Setup complete. Using torch 1.8.1+cu111 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [5]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


In [6]:
%cd ./
#after following the link above, recieve python code with these fields filled in
from roboflow import Roboflow
rf = Roboflow(api_key="QWencAPU4nMRzIBHlDhp")
project = rf.workspace("smart-pot").project("geranium")
dataset = project.version(3).download("yolov5")

/home/lab03/EvolveYOLO/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Geranium-3 in yolov5pytorch:: 100%|██████████| 2616/2616 [00:00<00:00, 7722.70it/s]


In [7]:
# curl -L "https://app.roboflow.com/ds/EbfTvnORki?key=IHIDfi7nN4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
import os

os.environ["dataset.location"] = "./datasets"

In [9]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names: ['Geranium', 'Geranium Gray Mold', 'Geranium Leaf Spot', 'Geranium Pest Damage']
nc: 4
train: Geranium-3/train/images
val: Geranium-3/valid/images
test: Geranium-3/test/images

## Count images on the Dataset

In [10]:
!find {dataset.location} -name *.jpg | wc -l
!find {dataset.location}/train -name *.jpg | wc -l
!find {dataset.location}/valid -name *.jpg | wc -l
!find {dataset.location}/test -name *.jpg | wc -l

1302
1140
108
54


# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [11]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [12]:
#this is the model configuration we will use for our tutorial 
%cat ./models/yolov5s.yaml
# %cat ./models/yolov5m.yaml
# %cat ./models/yolov5x.yaml
# %cat ./models/hub/yolov5l6.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [13]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [14]:
%%writetemplate ./models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [15]:
%cat ./data/hyp.scratch.yaml

# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  #

In [17]:
%cat ./utils/google_utils.py

# Google utils: https://cloud.google.com/storage/docs/reference/libraries

import os
import platform
import subprocess
import time
from pathlib import Path

import requests
import torch


def gsutil_getsize(url=''):
    # gs://bucket/file size https://cloud.google.com/storage/docs/gsutil/commands/du
    s = subprocess.check_output(f'gsutil du {url}', shell=True).decode('utf-8')
    return eval(s.split(' ')[0]) if len(s) else 0  # bytes


def attempt_download(file, repo='ultralytics/yolov5'):
    # Attempt file download if does not exist
    file = Path(str(file).strip().replace("'", '').lower())

    if not file.exists():
        try:
            response = requests.get(f'https://api.github.com/repos/{repo}/releases/tags/v5.0').json() # github api
            assets = [x['name'] for x in response['assets']]  # release assets, i.e. ['yolov5s.pt', 'yolov5m.pt', ...]
            tag = response['tag_name']  # i.e. 'v1.0'
        except:  # fallback plan
            assets = ['yolov5s.pt', 

In [18]:
# %cd ./
# !python train.py -h

In [19]:
# wandb.init(project="YOLOv5",
#            config={
#                "batch_size": 32,
#                "learning_rate": 0.01,
#                "dataset": "Geranium-2",
#            })

## metrics/mAP_0.5 0.33859

In [20]:
%%time

%%wandb

%cd ./
!python train.py \
--img 416 \
--batch 8 \
--epochs 10 \
--data {dataset.location}/data.yaml \
--cfg ./models/custom_yolov5s.yaml \
--weights yolov5s.pt \
--name yolov5s_geranium \
--cache \
--hyp "./data/hyp.scratch.yaml" \
# --resume \

/home/lab03/EvolveYOLO/yolov5
github: ⚠️ WARNING: code is out of date by 1118 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu111 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='/home/lab03/EvolveYOLO/yolov5/Geranium-3/data.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='./data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5l6_geranium', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5l6_geranium2', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at


     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       7/9    0.805G   0.04009   0.02591   0.03608    0.1021         6       416
               Class      Images     Targets           P           R      mAP@.5
                 all         108         197       0.331       0.239        0.12      0.0404

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       8/9    0.805G   0.03512   0.02491   0.03445   0.09448        12       416
               Class      Images     Targets           P           R      mAP@.5
                 all         108         197       0.289       0.203      0.0914      0.0307

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       9/9    0.805G   0.03234   0.02271   0.03586   0.09091         8       416
               Class      Images     Targets           P           R      mAP@.5
                 all         108         197       0.272       0.239       0.108  

# Hyperparameter Evolution (Optional. 300 scenarios)

In [ ]:
%%time

!python train.py \
--img 416 \
--batch 8 \
--epochs 10 \
--data {dataset.location}/data.yaml \
--cfg ./models/custom_yolov5s.yaml \
--weights yolov5s.pt \
--project YOLOv5 \
--name yolov5s_geranium_evolve \
--cache \
--hyp "./data/hyp.scratch.yaml" \
--evolve

github: ⚠️ WARNING: code is out of date by 1118 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu111 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='/home/lab03/EvolveYOLO/yolov5/Geranium-3/data.yaml', device='', entity=None, epochs=10, evolve=True, exist_ok=False, global_rank=-1, hyp='./data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='evolve', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/evolve', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', workers=8, world_size=1)
hyperparameters: lr0=0.0152, lrf=0.22969, momentum=0.73434, weight_decay=0.00019, warmup_epochs=1.63, warmup_momentum=0

       9/9    0.807G   0.01517   0.02541   0.02792    0.0685         6       416
               Class      Images     Targets           P           R      mAP@.5
                 all         108         197        0.44       0.193       0.143      0.0525
            Geranium         108         197        0.44       0.193       0.143      0.0525
10 epochs completed in 0.033 hours.

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      metrics/mAP_0.5 ▁▁▁▁▁▁▁▁▁█
wandb: metrics/mAP_0.5:0.95 ▁▁▁▁▁▁▁▁▁█
wandb:    metrics/precision ▁▁▁▁▁▁▁▁▁█
wandb:       metrics/recall ▁▁▁▁▁▁▁▁▁█
wandb:       train/box_loss █▆▄▃▂▂▂▂▂▁
wandb:       train/cls_loss █▅▃▂▂▂▁▂▁▁
wandb:       train/obj_loss ▅▆██▇▅▄▂▂▁
wandb:         val/box_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/cls_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/obj_loss ▁▁▁▁▁▁▁▁▁█
wandb:                x/lr0 ▁▄▆▇██▇▇▅▄
wandb:          

Model Summary: 283 layers, 7263185 parameters, 7263185 gradients, 16.9 GFLOPS

Transferred 258/370 items from yolov5s.pt
Scaled weight_decay = 0.0002
Optimizer groups: 62 .bias, 70 conv.weight, 59 other
wandb: Tracking run with wandb version 0.12.18
wandb: Run data is saved locally in /home/lab03/EvolveYOLO/yolov5/wandb/run-20220621_222628-f9xgbnrc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run evolve
wandb: ⭐️ View project at https://wandb.ai/angelajlee/YOLOv5
wandb: 🚀 View run at https://wandb.ai/angelajlee/YOLOv5/runs/f9xgbnrc
train: Scanning 'Geranium-3/train/labels.cache' for images and labels... 1140 fo
train: Caching images (0.6GB): 100%|███████| 1140/1140 [00:01<00:00, 747.31it/s]
val: Scanning 'Geranium-3/valid/labels.cache' for images and labels... 108 found

autoanchor: Analyzing anchors... anchors/target = 0.00, Best Possible Recall (BPR) = 0.0000. Attempting to improve anchors, please wait...
autoanchor: Running kmeans for 9 anchors on 1721 points...
va

  9                -1  1   1248768  models.common.BottleneckCSP             [512, 512, 1, False]          
 10                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]              
 11                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 12           [-1, 6]  1         0  models.common.Concat                    [1]                           
 13                -1  1    378624  models.common.BottleneckCSP             [512, 256, 1, False]          
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]              
 15                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 16           [-1, 4]  1         0  models.common.Concat                    [1]                           
 17                -1  1     95104  models.common.BottleneckCSP             [256, 128, 1, False]          
 18                -1  1    147712  m

Overriding model.yaml anchors with anchors=2.91435

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    161152  models.common.BottleneckCSP             [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    641792  models.common.BottleneckCSP             [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]

...

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      metrics/mAP_0.5 ▁▁▁▁▁▁▁▁▁█
wandb: metrics/mAP_0.5:0.95 ▁▁▁▁▁▁▁▁▁█
wandb:    metrics/precision ▁▁▁▁▁▁▁▁▁█
wandb:       metrics/recall ▁▁▁▁▁▁▁▁▁█
wandb:       train/box_loss █▆▄▃▂▂▂▂▁▁
wandb:       train/cls_loss █▅▃▂▂▂▂▁▁▁
wandb:       train/obj_loss ▄▅▇██▆▅▄▂▁
wandb:         val/box_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/cls_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/obj_loss ▁▁▁▁▁▁▁▁▁█
wandb:                x/lr0 ▁▄▆▇██▇▇▅▄
wandb:                x/lr1 ▁▄▆▇██▇▇▅▄
wandb:                x/lr2 █▇▆▅▄▂▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      metrics/mAP_0.5 0.14945
wandb: metrics/mAP_0.5:0.95 0.06227
wandb:    metrics/precision 0.41331
wandb:       metrics/recall 0.2335
wandb:       train/box_loss 0.01485
wandb:       train/cls_loss 0.02131
wandb:       train/obj_loss 0.02184
wandb:         val/box_loss 0.02749
wandb:  

       1/9    0.805G   0.03036   0.02886   0.03161   0.09083         3       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       2/9    0.805G   0.02314   0.03112   0.03104   0.08531         6       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       3/9    0.805G   0.02024   0.03048   0.03093   0.08164         4       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       4/9    0.805G   0.01936   0.02918   0.03004   0.07859         1       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       5/9    0.805G    0.0194   0.02826   0.02952   0.07718        10       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       6/9    0.805G   0.01955   0.02652   0.02865   0.07472         7       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       7/9    0.805G  

train: Scanning 'Geranium-3/train/labels.cache' for images and labels... 1140 fo
train: Caching images (0.6GB): 100%|██████| 1140/1140 [00:01<00:00, 1093.96it/s]
val: Scanning 'Geranium-3/valid/labels.cache' for images and labels... 108 found

autoanchor: Analyzing anchors... anchors/target = 0.00, Best Possible Recall (BPR) = 0.0000. Attempting to improve anchors, please wait...
autoanchor: Running kmeans for 9 anchors on 1721 points...
autoanchor: thr=0.37: 1.0000 best possible recall, 6.62 anchors past thr
autoanchor: n=9, img_size=416, metric_all=0.524/0.858-mean/best, past_thr=0.614-mean: 74,85,  115,126,  132,201,  214,161,  192,262,  222,350,  309,254,  314,381,  399,395
autoanchor: Evolving anchors with Genetic Algorithm: fitness = 0.8602: 100%|█| 1
autoanchor: thr=0.37: 1.0000 best possible recall, 6.62 anchors past thr
autoanchor: n=9, img_size=416, metric_all=0.527/0.860-mean/best, past_thr=0.618-mean: 72,92,  108,119,  130,196,  204,171,  194,259,  284,255,  230,360,  316,3

Model Summary: 283 layers, 7271276 parameters, 7271276 gradients, 17.0 GFLOPS

Transferred 258/370 items from yolov5s.pt
Scaled weight_decay = 0.0002
Optimizer groups: 62 .bias, 70 conv.weight, 59 other
wandb: Tracking run with wandb version 0.12.18
wandb: Run data is saved locally in /home/lab03/EvolveYOLO/yolov5/wandb/run-20220622_015723-2d9p66bq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run evolve
wandb: ⭐️ View project at https://wandb.ai/angelajlee/YOLOv5
wandb: 🚀 View run at https://wandb.ai/angelajlee/YOLOv5/runs/2d9p66bq
train: Scanning 'Geranium-3/train/labels.cache' for images and labels... 1140 fo
train: Caching images (0.6GB): 100%|██████| 1140/1140 [00:00<00:00, 1143.61it/s]
val: Scanning 'Geranium-3/valid/labels.cache' for images and labels... 108 found

autoanchor: Analyzing anchors... anchors/target = 0.00, Best Possible Recall (BPR) = 0.0000. Attempting to improve anchors, please wait...
autoanchor: Running kmeans for 12 anchors on 1721 points...
a

 24      [17, 20, 23]  1     16182  models.yolo.Detect                      [4, [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]], [128, 256, 512]]
Model Summary: 283 layers, 7255094 parameters, 7255094 gradients, 16.9 GFLOPS

Transferred 258/370 items from yolov5s.pt
Scaled weight_decay = 0.00025
Optimizer groups: 62 .bias, 70 conv.weight, 59 other
wandb: Tracking run with wandb version 0.12.18
wandb: Run data is saved locally in /home/lab03/EvolveYOLO/yolov5/wandb/run-20220622_015945-1cf1ej6d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run evolve
wandb: ⭐️ View project at https://wandb.ai/angelajlee/YOLOv5
wandb: 🚀 View run at https://wandb.ai/angelajlee/YOLOv5/runs/1cf1ej6d
train: Scanning 'Geranium-3/train/labels.cache' for images and labels... 1140 fo
train: Caching images (0.6GB): 100%|██████| 1140/1140 [00:01<00:00, 1108.03it/s]
val: Scanning 'Geranium-3/valid/labels.cache' for images and labels... 108 found

autoanchor: Analyzing anchors... anchors/target = 0.00, Be

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# !kill 17409

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# %load_ext tensorboard
# # %reload_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='./runs/train/yolov5l6_geranium3/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# print out an augmented training example / Mosaic Augmentation 적용된 train 이미지
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='./runs/train/yolov5l6_geranium3/train_batch0.jpg', width=900)

In [ ]:
# display our ground truth data / test 데이터 정답값
print("GROUND TRUTH TEST DATA:")
Image(filename='./runs/train/yolov5l6_geranium3/test_batch0_labels.jpg', width=900)

In [ ]:
# display our prediction / test 데이터 예측 결과값
print("PREDICT TEST DATA:")
Image(filename='./runs/train/yolov5l6_geranium3/test_batch0_pred.jpg', width=900)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls ./runs

In [ ]:
%ls ./runs/train/yolov5l6_geranium3/weights

In [ ]:
!python /home/lab03/jupyter_home/yolov5/detect.py -h

In [ ]:
from IPython.display import Image

!cd 
!python detect.py --weights /home/lab03/jupyter_home/yolov5/runs/train/yolov5l6_geranium3/weights/best.pt --img 256 --conf 0.4 --source /home/lab03/jupyter_home/yolov5/data/images/geraniumtest1.jpg
Image(filename='/home/lab03/jupyter_home/yolov5/runs/detect/exp/geraniumtest1.jpg', width=600)
# --conf : 라벨을 표시할 최소 확률 값

In [ ]:
from IPython.display import Image

!python detect.py --weights runs/train/yolov5l6_geranium3/weights/best.pt --img 256 --conf 0.4 --source data/images/geraniumtest1.jpg
Image(filename='./runs/detect/exp/geraniumtest1.jpg', width=600)
# --conf : 라벨을 표시할 최소 확률 값

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd ./
!python detect.py --weights runs/train/yolov5l6_geranium3/weights/best.pt --img 256 --conf 0.4 --source Geranium-2/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('./runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Computing the mAP on the test dataset

In [ ]:
%cd ./
!python test.py -h

In [ ]:
%cd ./
!python test.py --weights runs/train/yolov5l6_geranium3/weights/best.pt --data models/custom_yolov5l6.yaml --img 256 --name yolo_test_geranium --task 'test'

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
%cd ./
# %mkdir YOLO v5 Weights
%cp ./runs/train/yolov5l6_geranium3/weights/best.pt /home/lab03/jupyter_home/YOLOv5Weights/

## Congrats!

Hope you enjoyed this!

--Team [Roboflow](https://roboflow.ai)